In [2]:
# Importing the data
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
# Load data from CSV
file_path = r'/content/drive/MyDrive/PR/Advertising1.csv'
df = pd.read_csv(file_path)
print (df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   sample index     TV  radio  newspaper  sales
0             1  230.1   37.8       69.2   22.1
1             2   44.5   39.3       45.1   10.4
2             3   17.2   45.9       69.3    9.3
3             4  151.5   41.3       58.5   18.5
4             5  180.8   10.8       58.4   12.9


In [3]:
# Splitting the data into training and testing
from sklearn.model_selection import train_test_split

X = df[['TV', 'radio', 'newspaper']]
y = df['sales']

# using the train test split function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)

In [15]:
# Training a linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)

# Estimating the coefficients
intercept = model.intercept_
coefficients = model.coef_

print(f"intercept: {model.intercept_}")
print(f"coefficients: {model.coef_}")

intercept: 2.979067338122629
coefficients: [0.04472952 0.18919505 0.00276111]


In [16]:
# Calculating model statistics
from scipy.stats import t
import statsmodels.api as sm

# Fit a linear regression model on the training data
X_train_const = sm.add_constant(X_train)
sm_model = sm.OLS(y_train, X_train_const).fit()

# Get predictions for both training and testing data
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# RSS (Residual Sum of Squares)
rss_train = np.sum((y_train - y_pred_train) ** 2)
rss_test = np.sum((y_test - y_pred_test) ** 2)

#RSE (Residual Standard Error)
N_train, N_test = len(y_train), len(y_test)
d = X_train.shape[1] - 1  # Number of model parameters (excluding the intercept)
rse_train = np.sqrt(rss_train / (N_train - d))
rse_test = np.sqrt(rss_test / (N_test - d))

# MSE (Mean Squared Error)
mse_train = np.mean((y_train - y_pred_train) ** 2)
mse_test = np.mean((y_test - y_pred_test) ** 2)

# R-squared (R2) statistic
tss_train = np.sum((y_train - np.mean(y_train)) ** 2)
tss_test = np.sum((y_test - np.mean(y_test)) ** 2)

r2_train = 1 - (rss_train / tss_train)
r2_test = 1 - (rss_test / tss_test)

# Standard Error
# Training data
X_train_ = np.hstack((np.ones((N_train, 1)), X_train))
sigma2_train = rse_train**2
XTX_train = np.dot(X_train_.T, X_train_)
XTX_inv_train = np.linalg.inv(XTX_train)
se_train = np.sqrt(np.diagonal(sigma2_train * XTX_inv_train))
std_error_w0_train = se_train[0]
std_error_tv_train = se_train[1]
std_error_radio_train = se_train[2]
std_error_newspaper_train = se_train[3]

# Testing data
X_test_ = np.hstack((np.ones((N_test, 1)), X_test))
sigma2_test = rse_test**2
XTX_test = np.dot(X_test_.T, X_test_)
XTX_inv_test = np.linalg.inv(XTX_test)
se_test = np.sqrt(np.diagonal(sigma2_test * XTX_inv_test))
std_error_w0_test = se_test[0]
std_error_tv_test = se_test[1]
std_error_radio_test = se_test[2]
std_error_newspaper_test = se_test[3]

# t-statistic
# Training data
# Calculate the t-statistics for each feature in the training data
t_statistic_w0_train = intercept / std_error_w0_train
t_statistic_tv_train = coefficients[0] / std_error_tv_train
t_statistic_radio_train = coefficients[1] / std_error_radio_train
t_statistic_newspaper_train = coefficients[2] / std_error_newspaper_train

# Testing data
# Calculate the t-statistics for each feature in the training data
t_statistic_w0_test = intercept / std_error_w0_test
t_statistic_tv_test = coefficients[0] / std_error_tv_test
t_statistic_radio_test = coefficients[1] / std_error_radio_test
t_statistic_newspaper_test = coefficients[2] / std_error_newspaper_test

# p-value
# Training data
# Degrees of freedom
degrees_of_freedom_train = N_train - (d) # d is the number of model parameters

# Calculate the p-values for each feature in the training data
p_value_w0_train = 2 * (1 - t.cdf(np.abs(t_statistic_w0_train), df=degrees_of_freedom_train))
p_value_tv_train = 2 * (1 - t.cdf(np.abs(t_statistic_tv_train), df=degrees_of_freedom_train))
p_value_radio_train = 2 * (1 - t.cdf(np.abs(t_statistic_radio_train), df=degrees_of_freedom_train))
p_value_newspaper_train = 2 * (1 - t.cdf(np.abs(t_statistic_newspaper_train), df=degrees_of_freedom_train))

# Testing data
# Degrees of freedom
degrees_of_freedom_test = N_test - (d)  # d is the number of model parameters

# Calculate the p-values for each feature in the training data
p_value_w0_test = 2 * (1 - t.cdf(np.abs(t_statistic_w0_test), df=degrees_of_freedom_test))
p_value_tv_test = 2 * (1 - t.cdf(np.abs(t_statistic_tv_test), df=degrees_of_freedom_test))
p_value_radio_test = 2 * (1 - t.cdf(np.abs(t_statistic_radio_test), df=degrees_of_freedom_test))
p_value_newspaper_test = 2 * (1 - t.cdf(np.abs(t_statistic_newspaper_test), df=degrees_of_freedom_test))


In [17]:
# Displaying training data statistics
print("Training Data:")
print(f"\nRSS : {rss_train}")
print(f"RSE : {rse_train}")
print(f"MSE : {mse_train}")
print(f"R2 : {r2_train}")
print("\nStandard Errors : ")
print(f"intercept : {std_error_w0_train}")
print(f"TV : {std_error_tv_train}")
print(f"radio : {std_error_radio_train}")
print(f"newspaper : {std_error_newspaper_train}")
print("\nt-Statistics : ")
print(f"intercept : {t_statistic_w0_train}")
print(f"TV : {t_statistic_tv_train}")
print(f"radio : {t_statistic_radio_train}")
print(f"newspaper : {t_statistic_newspaper_train}")
print("\np-Values : ")
print(f"intercept : {p_value_w0_train}")
print(f"TV : {p_value_tv_train}")
print(f"radio : {p_value_radio_train}")
print(f"newspaper : {p_value_newspaper_train}")

Training Data:

RSS : 432.8207076930262
RSE : 1.6551046999139907
MSE : 2.705129423081414
R2 : 0.8957008271017818

Standard Errors : 
intercept : 0.3512719617827373
TV : 0.0015571104695243736
radio : 0.00963184161400032
newspaper : 0.007003206547849787

t-Statistics : 
intercept : 8.480800240940354
TV : 28.72597567363294
radio : 19.64266666920404
newspaper : 0.39426430200247997

p-Values : 
intercept : 1.509903313490213e-14
TV : 0.0
radio : 0.0
newspaper : 0.693917549816129


In [18]:
# Displaying testing data statistics
print("Testing Data:")
print(f"\nRSS : {rss_test}")
print(f"RSE : {rse_test}")
print(f"MSE : {mse_test}")
print(f"R2 : {r2_test}")
print("\nStandard Errors : ")
print(f"intercept : {std_error_w0_test}")
print(f"TV : {std_error_tv_test}")
print(f"radio : {std_error_radio_test}")
print(f"newspaper : {std_error_newspaper_test}")
print("\nt-Statistics : ")
print(f"intercept : {t_statistic_w0_test}")
print(f"TV : {t_statistic_tv_test}")
print(f"radio : {t_statistic_radio_test}")
print(f"newspaper : {t_statistic_newspaper_test}")
print("\np-Values : ")
print(f"intercept : {p_value_w0_test}")
print(f"TV : {p_value_tv_test}")
print(f"radio : {p_value_radio_test}")
print(f"newspaper : {p_value_newspaper_test}")

Testing Data:

RSS : 126.96389415904413
RSE : 1.8278826848155574
MSE : 3.1740973539761033
R2 : 0.899438024100912

Standard Errors : 
intercept : 0.7105934325996864
TV : 0.0032532468664630393
radio : 0.020046686274825445
newspaper : 0.01138375274950763

t-Statistics : 
intercept : 4.192365425083924
TV : 13.749192515890034
radio : 9.437722107317406
newspaper : 0.24254869216889985

p-Values : 
intercept : 0.00015892309072174093
TV : 2.220446049250313e-16
radio : 1.662869841823067e-11
newspaper : 0.809659422215083


In [11]:
# Validating manually calculated values
print(sm_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     446.6
Date:                Fri, 08 Sep 2023   Prob (F-statistic):           2.53e-76
Time:                        17:48:52   Log-Likelihood:                -306.64
No. Observations:                 160   AIC:                             621.3
Df Residuals:                     156   BIC:                             633.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.9791      0.354      8.427      0.0

In [6]:
# Allocating 25,000 dollars each for both television and radio advertising
print(f"Estimated sales by allocating 25,000 dollars each for both television and radio advertising: {sm_model.predict([1, 25, 25, 0])}")

# Allocating 50,000 dollars for television advertising
print(f"Estimated sales by allocating 50,000 dollars for television advertising : {sm_model.predict([1, 50, 0, 0])}")

# Allocating 50,000 dollars for television advertising
print(f"Estimated sales by allocating 50,000 dollars for television advertising: {sm_model.predict([1, 0, 50, 0])}")

Estimated sales by allocating 25,000 dollars each for both television and radio advertising: [8.82718163]
Estimated sales by allocating 50,000 dollars for television advertising : [5.21554321]
Estimated sales by allocating 50,000 dollars for television advertising: [12.43882005]
